<a href="https://colab.research.google.com/github/Troyanovsky/awesome-TTS-Colab/blob/main/Supertonic_TTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🗣️ Supertonic TTS Colab

## 📄 Description

This notebook runs **Supertonic**, an **ultra-lightweight, lightning-fast on-device text-to-speech system** powered by **ONNX Runtime**.  
With only **66M parameters**, it delivers **privacy-safe**, **near-instant** speech generation and handles complex text formats without preprocessing.

**Capabilities:**  
Extreme-Speed Inference, Lightweight Deployment, Natural Text Handling, Fully Local Processing, Flexible Runtime Backends

---

## How to use

* Run the setup cell to install dependencies and initialize ONNX Runtime  
* Edit the text prompt you wish to synthesize  
* Execute the remaining cells to generate and play audio

---

## ⚙️ Model Highlights

* ⚡ **167× faster-than-real-time** speech generation on consumer hardware  
* 🪶 **Compact 66M architecture** optimized for edge, mobile, and browser environments  
* 🔒 **Complete on-device processing** for privacy and zero network latency  
* 🎨 **Robust text normalization** with built-in handling of numbers, dates, currency, and abbreviations  
* 🧩 **Configurable inference pipeline** with adjustable steps and batching  
* 🌐 **Deploy anywhere** via ONNX across servers, browsers, and edge devices

---

## 🧠 Model Details

* **Base Architecture:** 66M lightweight TTS  
* **Runtime:** ONNX Runtime (multiple backend support)  
* **Text Handling:** Built-in normalization for complex expressions  
* **Licenses:**  
  * **Code:** MIT License  
  * **Model:** OpenRAIL-M License  
* **Format:** ONNX for flexible deployment

---

## 🔗 Resources

* **GitHub Repository:** https://github.com/supertone-inc/supertonic  
* **Model Availability:** https://huggingface.co/Supertone/supertonic

---

## 🎙️ Explore More TTS Models

Looking for more cutting-edge voice models?  
👉 Check out the full collection: [awesome-TTS-Colab](https://github.com/Troyanovsky/awesome-TTS-Colab)

## Voice Generation

In [1]:
import os
from pathlib import Path

BASE_DIR = Path("/content/supertonic")

if not BASE_DIR.exists():
    # Install git-lfs for Hugging Face model files
    !apt-get -y install git-lfs
    !git lfs install

    # Clone Supertonic repo
    %cd /content
    !git clone https://github.com/supertone-inc/supertonic.git

    # Download ONNX models & voice styles into /content/supertonic/assets
    %cd /content/supertonic
    !git clone https://huggingface.co/Supertone/supertonic assets

# Make sure we are in the Python subfolder (where requirements.txt lives)
%cd /content/supertonic/py

# Install Python dependencies (requirements.txt is here)
!pip install -r requirements.txt

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
Git LFS initialized.
/content
Cloning into 'supertonic'...
remote: Enumerating objects: 292, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 292 (delta 28), reused 24 (delta 24), pack-reused 242 (from 1)
Receiving objects: 100% (292/292), 512.04 KiB | 2.64 MiB/s, done.
Resolving deltas: 100% (135/135), done.
/content/supertonic
Cloning into 'assets'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 58 (delta 22), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (58/58), 531.42 KiB | 7.81 MiB/s, done.
Filtering content: 100% (4/4), 251.30 MiB | 109.83 MiB/s, done.
/content/supertonic/py
  

In [2]:
text_to_generate = "This is Supertonic running in Google Colab. Hello from your ultra-fast TTS system!"
voice_style_path = "assets/voice_styles/M1.json"  # e.g. M1.json, F1.json, etc.
use_gpu = False          # GPU not supported yet (must be False)
total_step = 5           # Higher = better quality, slower
speed = 1.05             # 0.9–1.5 recommended
save_dir = "results"     # Output folder

In [3]:
import soundfile as sf
from IPython.display import Audio, display

from helper import load_text_to_speech, timer, sanitize_filename, load_voice_style

onnx_dir = "assets/onnx"

# Prepare inputs
text_list = [text_to_generate]
voice_style_paths = [voice_style_path]

assert len(voice_style_paths) == len(text_list), "voice_style_paths and text_list must have same length"
bsz = len(voice_style_paths)

# Load TTS and voice style
text_to_speech = load_text_to_speech(onnx_dir, use_gpu)
style = load_voice_style(voice_style_paths, verbose=True)

Using CPU for inference
Loaded 1 voice styles


In [5]:
os.makedirs(save_dir, exist_ok=True)

with timer("Generating speech from text"):
    # Non-batch mode: synthesize a single text
    wav, duration = text_to_speech(text_list[0], style, total_step, speed)

# We generate only one sample here
b = 0
out_fname = f"{sanitize_filename(text_list[b], 20)}.wav"
trimmed = wav[b, : int(text_to_speech.sample_rate * duration[b].item())]
out_path = os.path.join(save_dir, out_fname)

sf.write(out_path, trimmed, text_to_speech.sample_rate)
print(f"\nSaved audio to: {out_path}")

Generating speech from text...
  -> Generating speech from text completed in 0.91 sec

Saved audio to: results/This_is_Supertonic_r.wav


In [ ]:
# Play audio in Colab
display(Audio(trimmed, rate=text_to_speech.sample_rate))